In [8]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import os
from datetime import datetime

def import_csv_to_mysql(host, database, user, password, csv_folder='csv_folder'):
    """
    Import CSV files into MySQL database with proper error handling and validation
    Args:
        host: MySQL host
        database: Database name
        user: MySQL username  
        password: MySQL password
        csv_folder: Path to folder containing CSV files
    """
    connection = None
    cursor = None
    
    try:
        # Establish connection
        connection = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            allow_local_infile=True  # Updated parameter name
        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            print(f"Connected to MySQL database: {database}")
            
            # Define import order considering foreign key constraints
            tables = [
                'agents',
                'neighborhoods', 
                'properties',
                'property_features',
                'price_history'
            ]
            
            total_records = 0
            
            for table in tables:
                csv_file = os.path.join(csv_folder, f'{table}.csv')
                
                # Check if CSV file exists
                if not os.path.exists(csv_file):
                    print(f"⚠️  Warning: {csv_file} not found, skipping...")
                    continue
                
                print(f"\n📥 Importing {table}.csv...")
                
                try:
                    # Read CSV with error handling
                    df = pd.read_csv(csv_file)
                    print(f"   Loaded {len(df)} records from CSV")
                    
                    if df.empty:
                        print(f"   ⚠️  CSV file is empty, skipping...")
                        continue
                    
                    # Replace NaN/NaT with None for SQL NULL
                    df = df.replace({pd.NaT: None})
                    df = df.where(pd.notnull(df), None)
                    
                    # Clean column names (remove spaces, special chars)
                    df.columns = [col.strip().replace(' ', '_').lower() for col in df.columns]
                    
                    # Prepare SQL statement
                    cols = '`,`'.join(df.columns)
                    placeholders = ','.join(['%s'] * len(df.columns))
                    sql = f"INSERT INTO `{table}` (`{cols}`) VALUES ({placeholders})"
                    
                    # Convert DataFrame to list of tuples for executemany
                    data_tuples = [tuple(x) for x in df.to_numpy()]
                    
                    # Insert data with batch processing for large files
                    batch_size = 1000
                    for i in range(0, len(data_tuples), batch_size):
                        batch = data_tuples[i:i + batch_size]
                        cursor.executemany(sql, batch)
                        connection.commit()
                        print(f"   ✓ Committed batch {i//batch_size + 1}/{(len(data_tuples)-1)//batch_size + 1}")
                    
                    total_records += len(df)
                    print(f"   ✅ Successfully imported {len(df)} rows into {table}")
                    
                except Exception as e:
                    print(f"   ❌ Error importing {table}: {str(e)}")
                    connection.rollback()  # Rollback on error
                    # Continue with next table instead of stopping completely
                    continue
            
            print(f"\n🎉 Import completed! Total records imported: {total_records}")
            
    except Error as e:
        print(f"❌ Database connection error: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    finally:
        # Properly close connections
        if cursor:
            cursor.close()
        if connection and connection.is_connected():
            connection.close()
            print("Database connection closed.")

def test_connection(host, database, user, password):
    """Test database connection before import"""
    try:
        connection = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        if connection.is_connected():
            print("✅ Database connection successful!")
            connection.close()
            return True
    except Error as e:
        print(f"❌ Connection failed: {e}")
        return False

def check_csv_files(csv_folder='.'):
    """Check if all required CSV files exist"""
    tables = ['agents', 'neighborhoods', 'properties', 'property_features', 'price_history']
    missing_files = []
    
    print("🔍 Checking for CSV files...")
    for table in tables:
        csv_file = os.path.join(csv_folder, f'{table}.csv')
        if os.path.exists(csv_file):
            # Get file size
            size = os.path.getsize(csv_file)
            print(f"   ✅ {table}.csv ({size:,} bytes)")
        else:
            print(f"   ❌ {table}.csv - NOT FOUND")
            missing_files.append(f"{table}.csv")
    
    return len(missing_files) == 0

# Main execution
if __name__ == "__main__":
    # Configuration - UPDATE THESE!
    config = {
        'host': 'localhost',
        'database': 'sa_real_estate', 
        'user': 'root',      # Change this!
        'password': 'Sql@12345',  # Change this!
        'csv_folder': 'csv_folder'             # Current directory
    }
    
    print("🚀 Starting CSV to MySQL Import...")
    print(f"📂 Looking for CSV files in: {os.path.abspath(config['csv_folder'])}")
    
    # Pre-flight checks
    if not check_csv_files(config['csv_folder']):
        print("\n⚠️  Some CSV files are missing. Please check the files above.")
        exit(1)
    
   # Replace lines 165-167 with this:
    if not test_connection(host=config['host'], 
                      database=config['database'], 
                      user=config['user'], 
                      password=config['password']):
        print("\n⚠️  Please check your database credentials and try again.")
    exit(1)
    
    # Confirm before proceeding
    response = input("\nProceed with import? (y/n): ").lower().strip()
    if response in ['y', 'yes']:
        print("\n" + "="*50)
        import_csv_to_mysql(**config)
    else:
        print("Import cancelled.")


🚀 Starting CSV to MySQL Import...
📂 Looking for CSV files in: C:\Users\rekgo\Desktop\2025\Projects\SQL project\csv_folder
🔍 Checking for CSV files...
   ✅ agents.csv (4,762 bytes)
   ✅ neighborhoods.csv (2,941 bytes)
   ✅ properties.csv (233,884 bytes)
   ✅ property_features.csv (99,560 bytes)
   ✅ price_history.csv (55,814 bytes)
✅ Database connection successful!

Proceed with import? (y/n): y

Connected to MySQL database: sa_real_estate

📥 Importing agents.csv...
   Loaded 50 records from CSV
   ✓ Committed batch 1/1
   ✅ Successfully imported 50 rows into agents

📥 Importing neighborhoods.csv...
   Loaded 52 records from CSV
   ✓ Committed batch 1/1
   ✅ Successfully imported 52 rows into neighborhoods

📥 Importing properties.csv...
   Loaded 2000 records from CSV
   ✓ Committed batch 1/2
   ✓ Committed batch 2/2
   ✅ Successfully imported 2000 rows into properties

📥 Importing property_features.csv...
   Loaded 2000 records from CSV
   ✓ Committed batch 1/2
   ✓ Committed batch 2/2